In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:

IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

n_classes = 4  
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

data_argumentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])


In [3]:

model = models.Sequential([
    layers.Input(shape=input_shape),
    resize_and_rescale,
    data_argumentation,
    layers.Conv2D(32, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation="relu"),  # Reduce el número de filtros
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(n_classes, activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [4]:



train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_datagen.flow_from_directory(
    'PlantVillage',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

val_datagen = ImageDataGenerator()
val_ds = val_datagen.flow_from_directory(
    'PlantVillage',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

train_ds = tf.data.Dataset.from_generator(
    lambda: train_ds,
    output_signature=(
        tf.TensorSpec(shape=(None, IMAGE_SIZE, IMAGE_SIZE, CHANNELS), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).cache().shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


val_ds = tf.data.Dataset.from_generator(
    lambda: val_ds,
    output_signature=(
        tf.TensorSpec(shape=(None, IMAGE_SIZE, IMAGE_SIZE, CHANNELS), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


Found 3251 images belonging to 4 classes.
Found 3251 images belonging to 4 classes.


In [5]:
class_names = ['Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']
class_counts = [944, 1107, 339, 861]
class_weights = compute_class_weight('balanced', classes=np.arange(len(class_names)), y=np.repeat(np.arange(len(class_names)), class_counts))
class_weights = dict(enumerate(class_weights))

print("Class Weights:", class_weights)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


Class Weights: {0: 0.8609639830508474, 1: 0.7341915085817525, 2: 2.3974926253687316, 3: 0.9439605110336817}


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

# Parámetros
IMAGE_SIZE = 256
BATCH_SIZE = 32

# Crear un generador de datos sin aumento de datos
simple_datagen = ImageDataGenerator()

simple_ds = simple_datagen.flow_from_directory(
    'PlantVillage',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

# Convertir a tf.data.Dataset y optimizar
simple_ds = tf.data.Dataset.from_generator(
    lambda: simple_ds,
    output_signature=(
        tf.TensorSpec(shape=(None, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).cache().shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Medir el tiempo por lote sin aumento de datos
start_time = time.time()

# Procesar un lote
for images, labels in simple_ds.take(1):
    pass

end_time = time.time()
time_per_batch_simple = end_time - start_time

print(f"Tiempo por lote sin aumento de datos: {time_per_batch_simple:.2f} segundos")

# Crear el generador de datos con aumento de datos
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_datagen.flow_from_directory(
    'PlantVillage',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

# Convertir a tf.data.Dataset y optimizar
train_ds = tf.data.Dataset.from_generator(
    lambda: train_ds,
    output_signature=(
        tf.TensorSpec(shape=(None, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).cache().shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Medir el tiempo por lote con aumento de datos
start_time = time.time()

# Procesar un lote
for images, labels in train_ds.take(1):
    pass

end_time = time.time()
time_per_batch_augmented = end_time - start_time

print(f"Tiempo por lote con aumento de datos: {time_per_batch_augmented:.2f} segundos")

num_batches = 0
for _ in train_ds.take(10):  
    num_batches += 1
    print(f"Lote {num_batches} procesado")

print(f"Número de lotes en el conjunto de datos de entrenamiento (estimado): {num_batches * (len(train_ds) // 10)}")

Found 3251 images belonging to 4 classes.


In [ ]:
num_batches = 0
for _ in train_ds.take(10):  
    num_batches += 1
    print(f"Lote {num_batches} procesado")

print(f"Número de lotes en el conjunto de datos de entrenamiento (estimado): {num_batches * (len(train_ds) // 10)}")



In [ ]:
for images, labels in train_ds.take(1):
    print(f"Forma de las imágenes: {images.shape}")
    print(f"Forma de las etiquetas: {labels.shape}")


In [ ]:



history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    class_weight=class_weights,
    callbacks=[early_stopping]
)



Class Weights: {0: 0.8609639830508474, 1: 0.7341915085817525, 2: 2.3974926253687316, 3: 0.9439605110336817}
Epoch 1/50


In [ ]:
model_version = 2
save_dir = r"C:\Users\candy\Documents\Python\PlantasEnf\modelo"  
os.makedirs(save_dir, exist_ok=True)  
model.save(os.path.join(save_dir, f"{model_version}Prueba.keras"))


test_loss, test_acc = model.evaluate(val_ds)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

def predict(model, img):
    img_array = tf.expand_dims(img, 0)  # Crear un lote
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = 100 * np.max(predictions[0])
    return predicted_class, confidence

for images, labels in val_ds.take(1):
    plt.figure(figsize=(10, 10))  # Ajustar el tamaño de la figura si es necesario
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[int(labels[i])]
        plt.title(f"Actual: {actual_class}, \n Enfermedad: {predicted_class}.\n Confidence: {confidence:.2f}%", fontsize=8)  # Reducir el tamaño de la fuente
        plt.axis("off")
    plt.show()